In [241]:
import csv
import pickle as pkl
import json
import ast
import jsonlines
from copy import deepcopy

In [206]:
zh_scenes = []
scene_ids = ['s07e17c08t1', 's07e17c07t2', 's07e17c09t0']
with open('data/raw_source/dialogue_zh/all_coref_data_en_zh.json', 'r') as f:
    data = json.load(f)
    for item in data:
        if item['scene_id'] in scene_ids:
            zh_scenes.append(item)

en_scenes = []
scene_ids = ['s07e17c08t', 's07e17c07t', 's07e17c09t']
with open('data/raw_source/dialogue_zh/dev_temp.pkl', 'rb') as f:
    data = pkl.load(f)
    for item in data:
        if item['scene_id'] in scene_ids:
            en_scenes.append(item)

correction_data = []
with open('data/raw_source/correction_dialogue_zh/dev.csv', 'r') as f:
    reader = csv.DictReader(f)
    for i, item in enumerate(reader):
        correction_data.append(item)
        print(i, item['Input.src_tokens'])

0 ['Hey', ',', 'you', 'busy', '?']
1 ['No', '.', 'What', "'s", 'up', '?']
2 ['Have', 'you', 'heard', 'back', 'from', 'Emily', '?']
3 ['Have', 'you', 'heard', 'back', 'from', 'Emily', '?']
4 ['I', 'have', '.']
5 ['And', 'I', "'m", 'afraid', 'she', 'does', "n't", 'think', 'you', "'re", 'right', 'for', 'her', '.']
6 ['And', 'I', "'m", 'afraid', 'she', 'does', "n't", 'think', 'you', "'re", 'right', 'for', 'her', '.']
7 ['And', 'I', "'m", 'afraid', 'she', 'does', "n't", 'think', 'you', "'re", 'right', 'for', 'her', '.']
8 ['And', 'I', "'m", 'afraid', 'she', 'does', "n't", 'think', 'you', "'re", 'right', 'for', 'her', '.']
9 ['I', 'give', 'you', 'one', 'simple', 'thing', 'to', 'do--', 'contact', 'a', 'complete', 'stranger', 'and', 'make', 'her', 'fall', 'in', 'love', 'with', 'me--', 'and', 'you', 'blow', 'it', '!']
10 ['I', 'give', 'you', 'one', 'simple', 'thing', 'to', 'do--', 'contact', 'a', 'complete', 'stranger', 'and', 'make', 'her', 'fall', 'in', 'love', 'with', 'me--', 'and', 'you', '

In [207]:
correction_dict = {
    "s07e17c08t1": {},
    "s07e17c07t2": {},
    "s07e17c09t0": {}
}

In [213]:
# Process the first scene
zh_scene = zh_scenes[0]
en_scene = en_scenes[0]
scene_correction_data = correction_data[:74]
for i in range(len(scene_correction_data)):
    sample = correction_data[i]
    src_tokens = ast.literal_eval(sample['Input.src_tokens'])
    tar_tokens = ast.literal_eval(sample['Input.tar_tokens'])
    original_alignment = json.loads(sample['Answer.alignment'])
    new_alignment = json.loads(sample['Input.config_obj'])['alignment']
    old_query = [-1, -1, -1]
    new_query = [-1, -1, -1]
    sent_id = -1
    temp = "".join(tar_tokens)
    for k, item in enumerate(zh_scene['sentences']):
        if temp==''.join(item):
            sent_id = k
            break
    if sent_id==-1:
        continue
    # Locate source token
    for j, item in enumerate(original_alignment):
        if True in item:
            start_idx = item.index(True)
            item.reverse()
            end_idx = len(item)-item.index(True)
            item.reverse()
            old_query = [sent_id, start_idx, end_idx]
    for j, item in enumerate(new_alignment):
        if True in item:
            start_idx = item.index(True)
            item.reverse()
            end_idx = len(item)-item.index(True)
            item.reverse()
            new_query = [sent_id, start_idx, end_idx]
    if new_query==[-1, -1, -1]:
        continue
    if old_query==new_query:
        continue
    correction_dict['s07e17c08t1'][tuple(old_query)] = tuple(new_query)

# # Process the first scene
# zh_scene = zh_scenes[1]
# en_scene = en_scenes[1]
# scene_correction_data = correction_data[74:119]
# for i in range(len(scene_correction_data)):
#     sample = correction_data[i]
#     src_tokens = ast.literal_eval(sample['Input.src_tokens'])
#     tar_tokens = ast.literal_eval(sample['Input.tar_tokens'])
#     original_alignment = json.loads(sample['Answer.alignment'])
#     new_alignment = json.loads(sample['Input.config_obj'])['alignment']
#     old_query = [-1, -1, -1]
#     new_query = [-1, -1, -1]
#     sent_id = -1
#     temp = "".join(tar_tokens)
#     for k, item in enumerate(zh_scene['sentences']):
#         if temp==''.join(item):
#             sent_id = k
#             break
#     if sent_id==-1:
#         continue
#     # Locate source token
#     for j, item in enumerate(original_alignment):
#         if True in item:
#             start_idx = item.index(True)
#             item.reverse()
#             end_idx = len(item)-item.index(True)
#             item.reverse()
#             old_query = [sent_id, start_idx, end_idx]
#     for j, item in enumerate(new_alignment):
#         if True in item:
#             start_idx = item.index(True)
#             item.reverse()
#             end_idx = len(item)-item.index(True)
#             item.reverse()
#             new_query = [sent_id, start_idx, end_idx]
#     if new_query==[-1, -1, -1]:
#         continue
#     if old_query==new_query:
#         continue
#     correction_dict['s07e17c07t2'][tuple(old_query)] = tuple(new_query)
#
# # Process the first scene
# zh_scene = zh_scenes[2]
# en_scene = en_scenes[2]
# scene_correction_data = correction_data[119:]
# for i in range(len(scene_correction_data)):
#     sample = correction_data[i]
#     src_tokens = ast.literal_eval(sample['Input.src_tokens'])
#     tar_tokens = ast.literal_eval(sample['Input.tar_tokens'])
#     original_alignment = json.loads(sample['Answer.alignment'])
#     new_alignment = json.loads(sample['Input.config_obj'])['alignment']
#     old_query = [-1, -1, -1]
#     new_query = [-1, -1, -1]
#     sent_id = -1
#     temp = "".join(tar_tokens)
#     for k, item in enumerate(zh_scene['sentences']):
#         if temp==''.join(item):
#             sent_id = k
#             break
#     if sent_id==-1:
#         continue
#     # Locate source token
#     for j, item in enumerate(original_alignment):
#         if True in item:
#             start_idx = item.index(True)
#             item.reverse()
#             end_idx = len(item)-item.index(True)
#             item.reverse()
#             old_query = [sent_id, start_idx, end_idx]
#     for j, item in enumerate(new_alignment):
#         if True in item:
#             start_idx = item.index(True)
#             item.reverse()
#             end_idx = len(item)-item.index(True)
#             item.reverse()
#             new_query = [sent_id, start_idx, end_idx]
#     if new_query==[-1, -1, -1]:
#         continue
#     if old_query==new_query:
#         continue
#     correction_dict['s07e17c09t0'][tuple(old_query)] = tuple(new_query)

In [212]:
print(correction_dict)

{'s07e17c08t1': {(5, 7, 8): (5, 7, 9), (7, 13, 17): (7, 15, 17), (11, 0, 1): (11, 0, 2), (11, 5, 7): (11, 4, 7), (16, 0, 3): (16, 0, 1)}, 's07e17c07t2': {}, 's07e17c09t0': {}}


In [210]:
for item in en_scene['answers']:
    print(item)
    print(en_scene['sentences'][item[0][0]][item[0][1]:item[0][2]])

[(0, 2, 3), [(1, 0, 1)]]
['Howard']
[(1, 2, 3), 'notPresent']
['What']
[(2, 2, 3), [(2, 0, 1)]]
['I']
[(2, 7, 9), 'notPresent']
['the', 'bathroom']
[(3, 2, 3), [(2, 0, 1)]]
['You']
[(3, 6, 8), 'notPresent']
['the', 'bathroom']
[(4, 2, 3), [(4, 0, 1)]]
['I']
[(4, 6, 7), [(3, 6, 8)]]
['it']
[(4, 8, 9), [(3, 6, 8)]]
['it']
[(4, 15, 20), 'notPresent']
['all', 'my', 'e', '-', 'mails']
[(4, 16, 17), [(4, 0, 1)]]
['my']
[(4, 21, 23), 'notPresent']
['the', 'toilet']
[(4, 26, 28), 'notPresent']
['a', 'seatbelt']
[(5, 4, 5), [(4, 21, 23)]]
['it']
[(6, 2, 3), [(6, 0, 1)]]
['I']
[(6, 4, 5), 'notPresent']
['that']
[(6, 7, 9), 'notPresent']
['safety', 'concerns']
[(6, 11, 13), 'notPresent']
['the', 'window']
[(6, 13, 16), 'notPresent']
['two', 'apple', 'juices']
[(8, 4, 5), [(7, 0, 1)]]
['you']
[(9, 2, 3), [(9, 0, 1)]]
['I']
[(9, 7, 10), 'notPresent']
['a', 'better', 'friend']
[(9, 12, 13), [(8, 0, 1)]]
['you']
[(9, 17, 19), 'notPresent']
['irritating', 'things']
[(11, 7, 9), 'notPresent']
['the', '

In [150]:
print(en_scene.keys())

dict_keys(['sentences', 'query_spans', 'answer_spans', 'WorkerId', 'scene_id', 'clusters_all', 'clusters_no_plural', 'answers'])


In [3]:
data = []
with open('data/raw_source/correction_dialogue_zh/pilot.csv', 'r') as f:
    reader = csv.DictReader(f)
    for item in reader:
        data.append(item)

In [4]:
print(len(data))

101


In [77]:
data = []
with open('data/raw_source/correction_dialogue_zh/dev.csv', 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for item in reader:
        data.append(item)

for i in range(len(data)):
    sample = data[i]
    source_sentence = ast.literal_eval(sample['Input.src_tokens'])
    target_sentence = ast.literal_eval(sample['Input.tar_tokens'])
    alignment = json.loads(sample['Answer.alignment'])

In [81]:
all_dev_test = []
# with open('data/raw_source/dialogue_zh/train_temp.pkl', 'rb') as f:
#     all_dev_test.extend(pkl.load(f))
# with open('data/raw_source/dialogue/dev_temp.pkl', 'rb') as f:
#     all_dev_test.extend(pkl.load(f))
with open('data/raw_source/dialogue/test_temp.pkl', 'rb') as f:
    all_dev_test.extend(pkl.load(f))

In [85]:
print(all_dev_test[0])

{'sentences': [['Penny', ':', 'There', 'you', 'go', '.', 'You', "'re", 'all', 'done', '!'], ['Amy', ':', 'That', 'was', "n't", 'so', 'bad', '.', 'Nice', '.', 'Boy', ',', 'if', 'my', 'mom', 'could', 'see', 'me', 'now', ',', 'she', "'d", 'lock', 'me', 'in', 'the', 'sin', 'closet', '.'], ['Bernadette', ':', 'That', "'s", 'a', 'joke', ',', 'right', '?'], ['Amy', ':', 'Actually', ',', 'the', 'joke', 'was', 'on', 'her', '.', 'I', 'could', 'still', 'see', 'the', 'TV', 'through', 'the', 'slats', '.'], ['Penny', ':', 'I', "'m", 'starting', 'to', 'see', 'why', 'you', 'and', 'your', 'mom', 'are', "n't", 'very', 'close', '.'], ['Bernadette', ':', 'Bet', 'she', "'d", 'still', 'tell', 'her', 'if', 'she', 'got', 'married', '.'], ['Penny', ':', 'You', "'re", 'not', 'gon', 'na', 'let', 'this', 'go', ',', 'are', 'you', '?'], ['Amy', ':', 'Well', ',', 'it', "'s", 'kind', 'of', 'sad', 'that', 'they', 'do', "n't", 'know', '.', 'I', 'mean', ',', 'they', 'love', 'Leonard', '.'], ['Penny', ':', 'Fine', ',', '

In [82]:
for i, instance in enumerate(all_dev_test):
    if i >= 3:
        continue
    sentences = instance['sentences']
    temp = []
    for sent in sentences:
        temp.append(" ".join(sent[sent.index(":")+1:]))

    for j in range(len(data)):
        sample = data[j]
        src_tokens = " ".join(sample['Input.src_tokens'])
        source_sentence = ast.literal_eval(sample['Input.src_tokens'])
        target_sentence = ast.literal_eval(sample['Input.tar_tokens'])
        alignment = json.loads(sample['Answer.alignment'])
        if src_tokens in temp:
            print(temp)

In [42]:
print(len(all_dev_test))

268


In [39]:
print(all_dev_test[0]['sentences'])

[['Sheldon', ':', 'If', 'a', 'photon', 'is', 'directed', 'through', 'a', 'plane', 'with', 'two', 'slits', 'in', 'it', 'and', 'either', 'is', 'observed', 'it', 'will', 'not', 'go', 'through', 'both', '.', 'If', 'unobserved', ',', 'it', 'will', '.', 'If', 'it', "'s", 'observed', 'after', 'it', 'left', 'the', 'plane', ',', 'before', 'it', 'hits', 'its', 'target', '...', 'it', 'will', 'not', 'have', 'gone', 'through', 'both', 'slits', '.'], ['Leonard', ':', 'Agreed', '.', 'What', "'s", 'your', 'point', '?'], ['Sheldon', ':', 'There', "'s", 'no', 'point', ',', 'I', 'just', 'think', 'it', "'s", 'a', 'good', 'idea', 'for', 'a', 'T', '-', 'shirt', '.'], ['Leonard', ':', 'Excuse', 'me', '.'], ['Receptionist', ':', 'Hang', 'on', '.'], ['Leonard', ':', 'One', 'across', 'is', 'Aegean', ',', 'eight', 'down', 'is', 'Nabokov', '.', 'Twenty', '-', 'six', 'across', 'is', 'MCM', '.', 'Fourteen', 'down', 'is', '...', 'Move', 'your', 'finger', '.', 'phylum', ',', 'which', 'makes', '14', 'across', 'Port', 

In [220]:
data = []
with open('data/raw_source/dialogue_zh/dev-test-batch1_zh.json', 'r') as f:
    reader = jsonlines.Reader(f)
    for item in reader:
        for x in item:
            data.append(x)

In [ ]:
correction_dict = {

}

In [234]:
for i in range(1):
    sample = data[i]
    sentences = sample['sentences']
    annotations = sample['annotations']
    for anno in annotations:
        query = anno['query']
        antecedents = anno['antecedents']
        print(query)
        print(sentences[query['sentenceIndex']])
        print(sentences[query['sentenceIndex']][query['startToken']: query['endToken']])
        print(antecedents)
        print()
    print("=="*50)

{'sentenceIndex': 0, 'startToken': 1, 'endToken': 2}
['嘿', '你', '忙', '吗', '?']
['你']
['Amy']

{'sentenceIndex': 1, 'startToken': 2, 'endToken': 4}
['不', '忙', '啥', '事', '?']
['啥', '事']
['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']

{'sentenceIndex': 2, 'startToken': 6, 'endToken': 7}
['E', 'm', 'i', 'l', 'y', '回', '你', '没', '?']
['你']
['Amy']

{'sentenceIndex': 2, 'startToken': 0, 'endToken': 5}
['E', 'm', 'i', 'l', 'y', '回', '你', '没', '?']
['E', 'm', 'i', 'l', 'y']
['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't']

{'sentenceIndex': 5, 'startToken': 4, 'endToken': 5}
['情', '况', '不', '妙', '她', '觉', '得', '你', '们', '俩', '不', '合', '适']
['她']
[{'sentenceIndex': 2, 'startToken': 0, 'endToken': 5}]

{'sentenceIndex': 5, 'startToken': 7, 'endToken': 9}
['情', '况', '不', '妙', '她', '觉', '得', '你', '们', '俩', '不', '合', '适']
['你', '们']
['Raj']

{'sentenceIndex': 6, 'startToken': 0, 'endToken': 1}
['我', '就', '让', '你', '帮', '一', '个', '忙', '联', '系', '一', '个', '陌', '生', '人', '并', '让', '她', '爱', '上',

In [237]:
correction_dict = {(5, 7, 9):(5, 7, 10), (11,4,7):(11,5,7)}
remove_dict = {(7,7,8):None, (8,22,24):None}

In [253]:
output = []
for i in range(1):
    sample = data[i]
    new_annotations = []
    for anno in annotations:
        new_anno = deepcopy(anno)
        temp = (anno['query']['sentenceIndex'], anno['query']['startToken'], anno['query']['endToken'])
        # if temp in remove_dict:
        #     continue
        # if temp in correction_dict:
        #     sentenceIndex, startToken, endToken = correction_dict[temp]
        #     new_anno['query']['sentenceIndex'] = sentenceIndex
        #     new_anno['query']['startToken'] = startToken
        #     new_anno['query']['endToken'] = endToken
        new_annotations.append(new_anno)
    sample['annotations'] = new_annotations
    output.append(sample)

In [254]:
with open('data/raw_source/sample_uncorrected/temp.json', 'w') as f:
    json.dump(output, f)

In [246]:
with open('data/raw_source/sample/temp.pkl', 'wb') as f:
    pkl.dump(sample, f)